In [6]:
import tensorflow as tf
import numpy as np


model_path = "/kaggle/input/model-new/EfficiantNet_b3_latest_TRIAL_nw.h5"
model = tf.keras.models.load_model(model_path)
print("Model loaded!")



test_dir = "/kaggle/input/lscidmr-augment-split/Splitting_Augmented/Splitting_Augmented/test"

IMG_SIZE = (224, 224)
BATCH = 1   

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False
)


def representative_data_gen():
    for images, _ in test_ds.take(200):   # 100–300 samples recommended
        images = tf.cast(images, tf.float32) / 255.0
        yield [images]



converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen


converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8


tflite_model = converter.convert()

tflite_path = "/kaggle/working/SatNetB3.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print("Saved satnetb3 model to:", tflite_path)


Model loaded!
Found 7880 files belonging to 8 classes.
INFO:tensorflow:Assets written to: /tmp/tmpr0kbq101/assets


INFO:tensorflow:Assets written to: /tmp/tmpr0kbq101/assets


Saved artifact at '/tmp/tmpr0kbq101'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  134774720914704: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134774720909328: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134774720909136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134774720909904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134774720914128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134774720908368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134774720907792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134774720910288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134774720910672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134774720907984: TensorSpec(shape=(), dtype=tf.resource, name=Non

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1765059480.514426      47 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1765059480.514472      47 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


Saved satnetb3 model to: /kaggle/working/SatNetB3.tflite


In [7]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from tensorflow.keras.applications.efficientnet import preprocess_input

tflite_model_path = "/kaggle/input/satnetb3/SatNetB3.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("TFLite model loaded!")



test_dir = "/kaggle/input/lscidmr-augment-split/Splitting_Augmented/Splitting_Augmented/test"
IMG_SIZE = (224, 224)
BATCH = 1

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False
)
class_names = test_ds.class_names
print("Classes:", class_names)


y_true = []
y_pred = []

for images, labels in tqdm(test_ds, desc="Evaluating TFLite Model"):
    
    img = images[0].numpy().astype(np.float32)
    img = np.expand_dims(img, axis=0)  # shape (1,224,224,3)

    # --- Correct preprocessing for EfficientNetB3 ---
    img = preprocess_input(img)

    # --- Run inference ---
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])[0]

    pred_class = np.argmax(output)
    true_class = labels.numpy()[0]

    y_pred.append(pred_class)
    y_true.append(true_class)



accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print("\n==============================")
print("TFLITE MODEL ACCURACY: {:.2f}%".format(accuracy * 100))
print("==============================")


TFLite model loaded!
Found 7880 files belonging to 8 classes.
Classes: ['Dersert', 'ExtratropicalCyclone', 'HighIceCloud', 'LowWaterCloud', 'Ocean', 'Snow', 'TropicalCyclone', 'Vegetation']


Evaluating TFLite Model: 100%|██████████| 7880/7880 [10:01<00:00, 13.10it/s]


TFLITE MODEL ACCURACY: 98.19%
